In [1]:
import numpy as np
import pandas as pd
import os
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from scipy.interpolate import UnivariateSpline
import statsmodels.api as sm
import matplotlib.pyplot as plt
import collections

pd.set_option("display.max_column", 999)
print(os.listdir("../input"))

['march-machine-learning-mania-2025']


In [2]:
DATA_PATH = '/kaggle/input/march-machine-learning-mania-2025/'

In [4]:
massey = pd.read_csv(DATA_PATH + "MMasseyOrdinals.csv")

tourney_results = pd.concat([
    pd.read_csv(DATA_PATH + "MNCAATourneyDetailedResults.csv"),
    pd.read_csv(DATA_PATH + "WNCAATourneyDetailedResults.csv"),
], ignore_index=True)

seeds = pd.concat([
    pd.read_csv(DATA_PATH + "MNCAATourneySeeds.csv"),
    pd.read_csv(DATA_PATH + "WNCAATourneySeeds.csv"),
], ignore_index=True)

regular_results = pd.concat([
    pd.read_csv(DATA_PATH + "MRegularSeasonDetailedResults.csv"),
    pd.read_csv(DATA_PATH + "WRegularSeasonDetailedResults.csv"),
], ignore_index=True)

In [5]:
regular_results_swap = regular_results[[
    'Season', 'DayNum', 'LTeamID', 'LScore', 'WTeamID', 'WScore', 'WLoc', 'NumOT', 
    'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 
    'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']]

regular_results_swap.loc[regular_results['WLoc'] == 'H', 'WLoc'] = 'A'
regular_results_swap.loc[regular_results['WLoc'] == 'A', 'WLoc'] = 'H'
regular_results.columns.values[6] = 'location'
regular_results_swap.columns.values[6] = 'location'

regular_results.columns = [x.replace('W','T1_').replace('L','T2_') for x in list(regular_results.columns)]
regular_results_swap.columns = [x.replace('L','T1_').replace('W','T2_') for x in list(regular_results.columns)]

regular_data = pd.concat([regular_results, regular_results_swap]).sort_index().reset_index(drop = True)

In [6]:
tourney_results = pd.concat([
    pd.read_csv(DATA_PATH + "MNCAATourneyDetailedResults.csv"),
    pd.read_csv(DATA_PATH + "WNCAATourneyDetailedResults.csv"),
], ignore_index=True)

seeds = pd.concat([
    pd.read_csv(DATA_PATH + "MNCAATourneySeeds.csv"),
    pd.read_csv(DATA_PATH + "WNCAATourneySeeds.csv"),
], ignore_index=True)

regular_results = pd.concat([
    pd.read_csv(DATA_PATH + "MRegularSeasonDetailedResults.csv"),
    pd.read_csv(DATA_PATH + "WRegularSeasonDetailedResults.csv"),
], ignore_index=True)

In [7]:
def prepare_data(df):
    dfswap = df[['Season', 'DayNum', 'LTeamID', 'LScore', 'WTeamID', 'WScore', 'WLoc', 'NumOT', 
    'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 
    'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']]

    dfswap.loc[df['WLoc'] == 'H', 'WLoc'] = 'A'
    dfswap.loc[df['WLoc'] == 'A', 'WLoc'] = 'H'
    df.columns.values[6] = 'location'
    dfswap.columns.values[6] = 'location'    
      
    df.columns = [x.replace('W','T1_').replace('L','T2_') for x in list(df.columns)]
    dfswap.columns = [x.replace('L','T1_').replace('W','T2_') for x in list(dfswap.columns)]

    output = pd.concat([df, dfswap]).reset_index(drop=True)
    output.loc[output.location=='N','location'] = '0'
    output.loc[output.location=='H','location'] = '1'
    output.loc[output.location=='A','location'] = '-1'
    output.location = output.location.astype(int)
    
    output['PointDiff'] = output['T1_Score'] - output['T2_Score']
    
    return output

regular_data = prepare_data(regular_results)
tourney_data = prepare_data(tourney_results)

In [9]:
boxscore_cols = ['T1_Score', 'T2_Score', 
        'T1_FGM', 'T1_FGA', 'T1_FGM3', 'T1_FGA3', 'T1_FTM', 'T1_FTA', 'T1_OR', 'T1_DR', 'T1_Ast', 'T1_TO', 'T1_Stl', 'T1_Blk', 'T1_PF', 
        'T2_FGM', 'T2_FGA', 'T2_FGM3', 'T2_FGA3', 'T2_FTM', 'T2_FTA', 'T2_OR', 'T2_DR', 'T2_Ast', 'T2_TO', 'T2_Stl', 'T2_Blk', 'T2_PF', 
        'PointDiff']

boxscore_cols = [
        'T1_FGM', 'T1_FGA', 'T1_FGM3', 'T1_FGA3', 'T1_OR', 'T1_Ast', 'T1_TO', 'T1_Stl', 'T1_PF', 
        'T2_FGM', 'T2_FGA', 'T2_FGM3', 'T2_FGA3', 'T2_OR', 'T2_Ast', 'T2_TO', 'T2_Stl', 'T2_Blk',  
        'PointDiff']

In [10]:
funcs = [np.mean]

season_statistics = regular_data.groupby(["Season", 'T1_TeamID'])[boxscore_cols].agg(funcs)
season_statistics = regular_data.groupby(["Season", 'T1_TeamID'])[boxscore_cols].agg(funcs).reset_index()
season_statistics.columns = [''.join(col).strip() for col in season_statistics.columns.values]

season_statistics_T1 = season_statistics.copy()
season_statistics_T2 = season_statistics.copy()

season_statistics_T1.columns = ["T1_" + x.replace("T1_","").replace("T2_","opponent_") for x in list(season_statistics_T1.columns)]
season_statistics_T2.columns = ["T2_" + x.replace("T1_","").replace("T2_","opponent_") for x in list(season_statistics_T2.columns)]
season_statistics_T1.columns.values[0] = "Season"
season_statistics_T2.columns.values[0] = "Season"

tourney_data = tourney_data[['Season', 'DayNum', 'T1_TeamID', 'T1_Score', 'T2_TeamID' ,'T2_Score']]
tourney_data = pd.merge(tourney_data, season_statistics_T1, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_data = pd.merge(tourney_data, season_statistics_T2, on = ['Season', 'T2_TeamID'], how = 'left')

last14days_stats_T1 = regular_data.loc[regular_data.DayNum>118].reset_index(drop=True)
last14days_stats_T1['win'] = np.where(last14days_stats_T1['PointDiff']>0,1,0)
last14days_stats_T1 = last14days_stats_T1.groupby(['Season','T1_TeamID'])['win'].mean().reset_index(name='T1_win_ratio_14d')

last14days_stats_T2 = regular_data.loc[regular_data.DayNum>118].reset_index(drop=True)
last14days_stats_T2['win'] = np.where(last14days_stats_T2['PointDiff']<0,1,0)
last14days_stats_T2 = last14days_stats_T2.groupby(['Season','T2_TeamID'])['win'].mean().reset_index(name='T2_win_ratio_14d')

tourney_data = pd.merge(tourney_data, last14days_stats_T1, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_data = pd.merge(tourney_data, last14days_stats_T2, on = ['Season', 'T2_TeamID'], how = 'left')

regular_season_effects = regular_data[['Season','T1_TeamID','T2_TeamID','PointDiff']].copy()
regular_season_effects['T1_TeamID'] = regular_season_effects['T1_TeamID'].astype(str)
regular_season_effects['T2_TeamID'] = regular_season_effects['T2_TeamID'].astype(str)
regular_season_effects['win'] = np.where(regular_season_effects['PointDiff']>0,1,0)
march_madness = pd.merge(seeds[['Season','TeamID']],seeds[['Season','TeamID']],on='Season')
march_madness.columns = ['Season', 'T1_TeamID', 'T2_TeamID']
march_madness.T1_TeamID = march_madness.T1_TeamID.astype(str)
march_madness.T2_TeamID = march_madness.T2_TeamID.astype(str)
regular_season_effects = pd.merge(regular_season_effects, march_madness, on = ['Season','T1_TeamID','T2_TeamID'])

<ipython-input-10-aae4d6a8822a>:3: FutureWarning: The provided callable <function mean at 0x7b07294e4a60> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  season_statistics = regular_data.groupby(["Season", 'T1_TeamID'])[boxscore_cols].agg(funcs)
<ipython-input-10-aae4d6a8822a>:4: FutureWarning: The provided callable <function mean at 0x7b07294e4a60> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  season_statistics = regular_data.groupby(["Season", 'T1_TeamID'])[boxscore_cols].agg(funcs).reset_index()


In [11]:
def team_quality(season):
    formula = 'win~-1+T1_TeamID+T2_TeamID'
    glm = sm.GLM.from_formula(formula=formula, 
                              data=regular_season_effects.loc[regular_season_effects.Season==season,:], 
                              family=sm.families.Binomial()).fit()
    
    quality = pd.DataFrame(glm.params).reset_index()
    quality.columns = ['TeamID','quality']
    quality['Season'] = season
    #quality['quality'] = np.exp(quality['quality'])
    quality = quality.loc[quality.TeamID.str.contains('T1_')].reset_index(drop=True)
    quality['TeamID'] = quality['TeamID'].apply(lambda x: x[10:14]).astype(int)
    return quality

In [13]:
formula = 'win~-1+T1_TeamID+T2_TeamID'
glm = sm.GLM.from_formula(formula=formula, 
                          data=regular_season_effects.loc[regular_season_effects.Season==2010,:], 
                          family=sm.families.Binomial()).fit()

quality = pd.DataFrame(glm.params).reset_index()

glm_quality = pd.concat([team_quality(2010),
                         team_quality(2011),
                         team_quality(2012),
                         team_quality(2013),
                         team_quality(2014),
                         team_quality(2015),
                         team_quality(2016),
                         team_quality(2017),
                         team_quality(2018),
                         team_quality(2019),
                         ##team_quality(2020),
                         team_quality(2021),
                         team_quality(2022),
                         team_quality(2023),
                         team_quality(2024)
                         ]).reset_index(drop=True)

glm_quality_T1 = glm_quality.copy()
glm_quality_T2 = glm_quality.copy()
glm_quality_T1.columns = ['T1_TeamID','T1_quality','Season']
glm_quality_T2.columns = ['T2_TeamID','T2_quality','Season']

/usr/local/lib/python3.10/dist-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/usr/local/lib/python3.10/dist-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/usr/local/lib/python3.10/dist-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/usr/local/lib/python3.10/dist-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/usr/local/lib/python3.10/dist-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/usr/local/lib/python3.10/dist-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/usr/local/lib/python3.10/dist-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)

In [14]:
tourney_data = pd.merge(tourney_data, glm_quality_T1, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_data = pd.merge(tourney_data, glm_quality_T2, on = ['Season', 'T2_TeamID'], how = 'left')

seeds['seed'] = seeds['Seed'].apply(lambda x: int(x[1:3]))
seeds_T1 = seeds[['Season','TeamID','seed']].copy()
seeds_T2 = seeds[['Season','TeamID','seed']].copy()
seeds_T1.columns = ['Season','T1_TeamID','T1_seed']
seeds_T2.columns = ['Season','T2_TeamID','T2_seed']

tourney_data = pd.merge(tourney_data, seeds_T1, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_data = pd.merge(tourney_data, seeds_T2, on = ['Season', 'T2_TeamID'], how = 'left')
tourney_data["Seed_diff"] = tourney_data["T1_seed"] - tourney_data["T2_seed"]

In [15]:
tourney_data['IsMensTeam'] = (tourney_data['T1_TeamID'] < 1500).astype(int)

latest_ranking_days = massey.groupby('Season')['RankingDayNum'].apply(
    lambda x: x[x < 133].max() if any(x < 133) else x.max()
).reset_index()
latest_ranking_days.rename(columns={'RankingDayNum': 'LatestRankingDay'}, inplace=True)

massey_latest = pd.merge(massey, latest_ranking_days, on='Season')
massey_latest = massey_latest[massey_latest['RankingDayNum'] == massey_latest['LatestRankingDay']]

key_systems = ['AP', 'KEN', 'SAG', 'MOR', 'POM', 'RTH', 'DOL', 'RPI']

for system in key_systems:
    system_ranks = massey_latest[massey_latest['SystemName'] == system].copy()
    
    if len(system_ranks) > 0:
        system_ranks_T1 = system_ranks[['Season', 'TeamID', 'OrdinalRank']].copy()
        system_ranks_T2 = system_ranks[['Season', 'TeamID', 'OrdinalRank']].copy()
        
        system_ranks_T1.columns = ['Season', 'T1_TeamID', f'T1_{system}_Rank']
        system_ranks_T2.columns = ['Season', 'T2_TeamID', f'T2_{system}_Rank']
        
        tourney_data = pd.merge(tourney_data, system_ranks_T1, on=['Season', 'T1_TeamID'], how='left')
        tourney_data = pd.merge(tourney_data, system_ranks_T2, on=['Season', 'T2_TeamID'], how='left')
        
        tourney_data[f'{system}_Rank_diff'] = tourney_data[f'T1_{system}_Rank'] - tourney_data[f'T2_{system}_Rank']

avg_ranks = massey_latest.groupby(['Season', 'TeamID'])['OrdinalRank'].mean().reset_index()
avg_ranks.columns = ['Season', 'TeamID', 'AvgRank']

avg_ranks_T1 = avg_ranks.copy()
avg_ranks_T2 = avg_ranks.copy()
avg_ranks_T1.columns = ['Season', 'T1_TeamID', 'T1_AvgRank']
avg_ranks_T2.columns = ['Season', 'T2_TeamID', 'T2_AvgRank']

tourney_data = pd.merge(tourney_data, avg_ranks_T1, on=['Season', 'T1_TeamID'], how='left')
tourney_data = pd.merge(tourney_data, avg_ranks_T2, on=['Season', 'T2_TeamID'], how='left')

tourney_data['AvgRank_diff'] = tourney_data['T1_AvgRank'] - tourney_data['T2_AvgRank']

In [16]:
rank_std = massey_latest.groupby(['Season', 'TeamID'])['OrdinalRank'].std().reset_index()
rank_std.columns = ['Season', 'TeamID', 'RankStd']

rank_std_T1 = rank_std.copy()
rank_std_T2 = rank_std.copy()
rank_std_T1.columns = ['Season', 'T1_TeamID', 'T1_RankStd']
rank_std_T2.columns = ['Season', 'T2_TeamID', 'T2_RankStd']

tourney_data = pd.merge(tourney_data, rank_std_T1, on=['Season', 'T1_TeamID'], how='left')
tourney_data = pd.merge(tourney_data, rank_std_T2, on=['Season', 'T2_TeamID'], how='left')

rank_count = massey_latest.groupby(['Season', 'TeamID']).size().reset_index(name='NumRankings')
rank_count_T1 = rank_count.copy()
rank_count_T2 = rank_count.copy()
rank_count_T1.columns = ['Season', 'T1_TeamID', 'T1_NumRankings']
rank_count_T2.columns = ['Season', 'T2_TeamID', 'T2_NumRankings']

tourney_data = pd.merge(tourney_data, rank_count_T1, on=['Season', 'T1_TeamID'], how='left')
tourney_data = pd.merge(tourney_data, rank_count_T2, on=['Season', 'T2_TeamID'], how='left')

In [17]:
previous_days = massey.groupby(['Season', 'SystemName'])['RankingDayNum'].apply(
    lambda x: x.sort_values().iloc[[-2, -1]].reset_index(drop=True) if len(x) >= 2 else pd.Series([x.iloc[0], x.iloc[0]])
).reset_index()
previous_days.columns = ['Season', 'SystemName', 'idx', 'RankingDayNum']
previous_days = previous_days.pivot(index=['Season', 'SystemName'], columns='idx', values='RankingDayNum').reset_index()
previous_days.columns = ['Season', 'SystemName', 'PreviousRankingDay', 'LastRankingDay']

previous_massey = pd.merge(massey, previous_days, on=['Season', 'SystemName'])
previous_massey = previous_massey[previous_massey['RankingDayNum'] == previous_massey['PreviousRankingDay']]
previous_massey = previous_massey[['Season', 'SystemName', 'TeamID', 'OrdinalRank']]
previous_massey.columns = ['Season', 'SystemName', 'TeamID', 'PreviousRank']

last_massey = pd.merge(massey, previous_days, on=['Season', 'SystemName'])
last_massey = last_massey[last_massey['RankingDayNum'] == last_massey['LastRankingDay']]
last_massey = last_massey[['Season', 'SystemName', 'TeamID', 'OrdinalRank']]
last_massey.columns = ['Season', 'SystemName', 'TeamID', 'LastRank']

momentum = pd.merge(previous_massey, last_massey, on=['Season', 'SystemName', 'TeamID'])
momentum['RankChange'] = momentum['PreviousRank'] - momentum['LastRank']  # Positive means improvement

avg_momentum = momentum.groupby(['Season', 'TeamID'])['RankChange'].mean().reset_index()
avg_momentum.columns = ['Season', 'TeamID', 'AvgMomentum']

avg_momentum_T1 = avg_momentum.copy()
avg_momentum_T2 = avg_momentum.copy()
avg_momentum_T1.columns = ['Season', 'T1_TeamID', 'T1_AvgMomentum']
avg_momentum_T2.columns = ['Season', 'T2_TeamID', 'T2_AvgMomentum']

tourney_data = pd.merge(tourney_data, avg_momentum_T1, on=['Season', 'T1_TeamID'], how='left')
tourney_data = pd.merge(tourney_data, avg_momentum_T2, on=['Season', 'T2_TeamID'], how='left')
tourney_data['Momentum_diff'] = tourney_data['T1_AvgMomentum'] - tourney_data['T2_AvgMomentum']

In [18]:
massey_feature_cols = ['T1_AvgRank', 'T2_AvgRank', 'AvgRank_diff', 
                        'T1_RankStd', 'T2_RankStd',
                        'T1_NumRankings', 'T2_NumRankings',
                        'T1_AvgMomentum', 'T2_AvgMomentum', 'Momentum_diff']

for system in key_systems:
    if f'T1_{system}_Rank' in tourney_data.columns:
        massey_feature_cols.extend([f'T1_{system}_Rank', f'T2_{system}_Rank', f'{system}_Rank_diff'])

for col in massey_feature_cols:
    if col in tourney_data.columns:
        tourney_data[f'{col}_mens'] = tourney_data[col] * tourney_data['IsMensTeam']
    
base_features = list(season_statistics_T1.columns[2:999]) + \
    list(season_statistics_T2.columns[2:999]) + \
    list(seeds_T1.columns[2:999]) + \
    list(seeds_T2.columns[2:999]) + \
    list(last14days_stats_T1.columns[2:999]) + \
    list(last14days_stats_T2.columns[2:999]) + \
    ["Seed_diff"] + ["T1_quality", "T2_quality"]

massey_mens_features = [f'{col}_mens' for col in massey_feature_cols if f'{col}_mens' in tourney_data.columns]

In [19]:
features = base_features + massey_mens_features

y = tourney_data['T1_Score'] - tourney_data['T2_Score']

In [20]:
features = list(season_statistics_T1.columns[2:999]) + \
    list(season_statistics_T2.columns[2:999]) + \
    list(seeds_T1.columns[2:999]) + \
    list(seeds_T2.columns[2:999]) + \
    list(last14days_stats_T1.columns[2:999]) + \
    list(last14days_stats_T2.columns[2:999]) + \
    ["Seed_diff"] + ["T1_quality","T2_quality"]

In [21]:
X = tourney_data[features].values
dtrain = xgb.DMatrix(X, label = y)

In [22]:
def cauchyobj(preds, dtrain):
    labels = dtrain.get_label()
    c = 5000 
    x =  preds-labels    
    grad = x / (x**2/c**2+1)
    hess = -c**2*(x**2-c**2)/(x**2+c**2)**2
    return grad, hess

In [23]:
param = {} 
#param['objective'] = 'reg:linear'
param['eval_metric'] =  'mae'
param['booster'] = 'gbtree'
param['eta'] = 0.02 #change to ~0.02 for final run
param['subsample'] = 0.35
param['colsample_bytree'] = 0.7
param['num_parallel_tree'] = 10 #recommend 10
param['min_child_weight'] = 40
param['gamma'] = 10
param['max_depth'] =  3
param['silent'] = 1

print(param)

{'eval_metric': 'mae', 'booster': 'gbtree', 'eta': 0.02, 'subsample': 0.35, 'colsample_bytree': 0.7, 'num_parallel_tree': 10, 'min_child_weight': 40, 'gamma': 10, 'max_depth': 3, 'silent': 1}


In [24]:
xgb_cv = []
repeat_cv = 10 # recommend 10

for i in range(repeat_cv): 
    print(f"Fold repeater {i}")
    xgb_cv.append(
        xgb.cv(
          params = param,
          dtrain = dtrain,
          obj = cauchyobj,
          num_boost_round = 3000,
          folds = KFold(n_splits = 5, shuffle = True, random_state = i),
          early_stopping_rounds = 25,
          verbose_eval = 50
        )
    )

Fold repeater 0
[0]	train-mae:13.57403+0.05891	test-mae:13.57528+0.23801


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:15:46] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[50]	train-mae:10.43116+0.04229	test-mae:10.53994+0.21149
[100]	train-mae:9.64440+0.03956	test-mae:9.85037+0.17504
[150]	train-mae:9.36710+0.03936	test-mae:9.66491+0.17098
[200]	train-mae:9.21539+0.03992	test-mae:9.59929+0.17247
[250]	train-mae:9.10076+0.03969	test-mae:9.56934+0.17291
[300]	train-mae:8.99963+0.04028	test-mae:9.55189+0.17403
[350]	train-mae:8.90909+0.04034	test-mae:9.54542+0.17252
[400]	train-mae:8.82478+0.04063	test-mae:9.53933+0.17267
[450]	train-mae:8.74394+0.04146	test-mae:9.53533+0.17438
[481]	train-mae:8.69652+0.04095	test-mae:9.53598+0.17632
Fold repeater 1
[0]	train-mae:13.57349+0.07711	test-mae:13.57489+0.30751


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:16:22] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[50]	train-mae:10.43057+0.04889	test-mae:10.54222+0.21314
[100]	train-mae:9.64325+0.04605	test-mae:9.85107+0.19755
[150]	train-mae:9.36534+0.04582	test-mae:9.66185+0.20455
[200]	train-mae:9.21196+0.04498	test-mae:9.59720+0.21434
[250]	train-mae:9.09789+0.04652	test-mae:9.56963+0.22422
[300]	train-mae:8.99867+0.04830	test-mae:9.55792+0.23323
[350]	train-mae:8.90827+0.04944	test-mae:9.55156+0.23697
[400]	train-mae:8.82357+0.04986	test-mae:9.54671+0.23953
[450]	train-mae:8.74297+0.05080	test-mae:9.54626+0.24410
[463]	train-mae:8.72281+0.05099	test-mae:9.54701+0.24561
Fold repeater 2
[0]	train-mae:13.57356+0.05842	test-mae:13.57545+0.23750


...

[50]	train-mae:10.43214+0.05813	test-mae:10.54495+0.26795
[100]	train-mae:9.64390+0.06078	test-mae:9.86023+0.27673
[150]	train-mae:9.36361+0.06178	test-mae:9.67830+0.27257
[200]	train-mae:9.21017+0.06484	test-mae:9.61453+0.27245
[250]	train-mae:9.09555+0.06528	test-mae:9.58898+0.26827
[300]	train-mae:8.99581+0.06421	test-mae:9.57788+0.26494
[350]	train-mae:8.90347+0.06420	test-mae:9.57085+0.26036
[395]	train-mae:8.82594+0.06320	test-mae:9.57010+0.25804
Fold repeater 8
[0]	train-mae:13.57350+0.10118	test-mae:13.57477+0.41032


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:20:34] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[50]	train-mae:10.43286+0.08195	test-mae:10.53502+0.37788
[100]	train-mae:9.64626+0.08270	test-mae:9.83942+0.36618
[150]	train-mae:9.36675+0.08623	test-mae:9.66146+0.35719
[200]	train-mae:9.21237+0.08850	test-mae:9.60237+0.35247
[250]	train-mae:9.09523+0.08888	test-mae:9.58210+0.35190
[300]	train-mae:8.99499+0.09012	test-mae:9.57561+0.35039
[350]	train-mae:8.90294+0.09060	test-mae:9.56940+0.35017
[394]	train-mae:8.82633+0.09008	test-mae:9.56799+0.34749
Fold repeater 9
[0]	train-mae:13.57363+0.06382	test-mae:13.57535+0.25157


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:21:04] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[50]	train-mae:10.43526+0.05041	test-mae:10.53367+0.21643
[100]	train-mae:9.64603+0.04569	test-mae:9.84231+0.19757
[150]	train-mae:9.36983+0.04319	test-mae:9.65425+0.19262
[200]	train-mae:9.21685+0.04097	test-mae:9.58753+0.18207
[250]	train-mae:9.10292+0.03981	test-mae:9.56074+0.17764
[300]	train-mae:9.00315+0.04097	test-mae:9.55257+0.17605
[350]	train-mae:8.91102+0.03973	test-mae:9.54727+0.17893
[400]	train-mae:8.82564+0.03955	test-mae:9.54642+0.17970
[408]	train-mae:8.81210+0.03935	test-mae:9.54673+0.17999


In [25]:
iteration_counts = [np.argmin(x['test-mae-mean'].values) for x in xgb_cv]
val_mae = [np.min(x['test-mae-mean'].values) for x in xgb_cv]
iteration_counts, val_mae

([457, 438, 462, 389, 440, 637, 450, 371, 370, 384],
 [9.5352501637851,
  9.545600728284343,
  9.525169485470645,
  9.56030076817667,
  9.544954531453138,
  9.518652758123725,
  9.559955355203034,
  9.56921682526018,
  9.567636328985348,
  9.544987312163048])

In [26]:
oof_preds = []
for i in range(repeat_cv):
    print(f"Fold repeater {i}")
    preds = y.copy()
    kfold = KFold(n_splits = 5, shuffle = True, random_state = i)    
    for train_index, val_index in kfold.split(X,y):
        dtrain_i = xgb.DMatrix(X[train_index], label = y[train_index])
        dval_i = xgb.DMatrix(X[val_index], label = y[val_index])  
        model = xgb.train(
              params = param,
              dtrain = dtrain_i,
              num_boost_round = iteration_counts[i],
              verbose_eval = 50
        )
        preds[val_index] = model.predict(dval_i)
    oof_preds.append(np.clip(preds,-30,30))

Fold repeater 0


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:22:10] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)
<ipython-input-26-477994df21d8>:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 2.19902306e+01  1.24678078e+01  5.32899904e+00  2.09548187e+01
  1.07806597e+01 -7.12072432e-01  1.04159784e+01 -8.71903801e+00
 -3.52902603e+00 -1.35922283e-01 -3.50643849e+00 -1.36244106e+00
  5.67299795e+00  2.85753822e+00  1.82981796e+01  3.93462396e+00
  2.69727492e+00  4.37359810e-01 -1.39862609e+00  7.94504976e+00
  2.03151798e+01 -7.47154474e-01  3.79337049e+00  3.03327389e+01
  9.20329475e+00 -1.76556289e+00  1.78650742e+01  6.79344416e+00
  1.11748466e+01  3.95318246e+00  1.32144547e+01 -6.64903045e-01
  4.90721750e+00  3.46282077e+00 -3.30636883e+00  2.22962093e+01
 -6.86720133e+00  1.54064922e+01  3.07419491e+00 

Fold repeater 1


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:22:39] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)
<ipython-input-26-477994df21d8>:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 1.89596577e+01  1.80486984e+01  3.75317121e+00  2.19804993e+01
 -2.13062692e+00  1.66388836e+01  2.18063984e+01  1.39392653e+01
  1.39623060e+01  2.80292249e+00  8.76191711e+00 -2.00363731e+00
 -3.87179613e+00 -6.25944185e+00 -3.32603025e+00  3.19266915e+00
  1.29153633e+01  1.98962002e+01  1.10727000e+00  1.34898543e+00
  6.84674978e+00  7.15912008e+00 -7.79842186e+00 -3.27516198e-01
  6.34845829e+00  4.92432165e+00 -6.65064096e-01  2.80116725e+00
 -1.95700958e-01 -3.44499826e+00  4.83765697e+00 -3.69480324e+00
  1.31893361e+00  2.40197525e+01  1.92824421e+01  1.54287605e+01
  4.75604439e+00  3.51030684e+00  5.34647417e+00 

Fold repeater 2


...

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:25:33] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)
<ipython-input-26-477994df21d8>:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-3.32153749e+00  1.03038430e+00  2.35420399e+01  4.32294488e-01
  1.54440765e+01 -1.23407173e+00 -2.02877712e+00  8.86695385e+00
  5.19505405e+00  5.85094976e+00 -1.79809856e+00 -3.84784436e+00
  2.37699814e+01  2.81739259e+00  4.14771986e+00  7.73533535e+00
  7.84413290e+00  3.99500728e+00 -1.62867737e+00  2.22486091e+00
  3.85601044e+00  8.02293968e+00  3.81889081e+00  2.50315690e+00
  1.23838024e+01  1.42872095e+01  3.66735792e+00 -1.57022295e+01
  2.13006172e+01 -8.76636982e-01  1.93752079e+01  3.38640976e+00
  6.75912762e+00  1.33261538e+01  5.00713396e+00  1.07497206e+01
 -2.71689725e+00  1.17771463e+01  8.41227150e+00 

Fold repeater 8


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:25:57] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)
<ipython-input-26-477994df21d8>:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 2.06906052e+01 -8.22424126e+00  1.73368988e+01 -8.65764046e+00
  1.85897236e+01  2.43578568e+01 -1.05565310e+00 -2.20317936e+00
  7.53826761e+00  5.33613682e+00  7.50138569e+00  4.63872862e+00
 -4.93868923e+00 -3.75713682e+00 -2.97220796e-01 -3.98798251e+00
  7.25423157e-01  1.35331011e+01  6.46521759e+00  2.28924580e+01
 -6.03503656e+00  4.72919941e+00  3.46565795e+00  4.73070717e+00
  2.75950265e+00  3.70985419e-01 -2.17707539e+00 -4.83158541e+00
  1.47060070e+01  5.79361820e+00  2.68255424e+01 -8.52707863e+00
  1.12983112e+01 -7.04530764e+00 -4.69195843e+00  4.39428329e+00
  3.65197039e+00  5.19252491e+00  8.33417511e+00 

Fold repeater 9


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:26:20] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)
<ipython-input-26-477994df21d8>:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 4.60959101e+00  2.09979534e+01 -7.09922314e+00  1.80520935e+01
  2.23493919e+01  1.40105021e+00 -3.26974988e+00 -3.71026611e+00
  5.40158892e+00  4.89811182e+00  4.04387093e+00  2.46105499e+01
  2.33669682e+01  8.22405434e+00 -2.34824038e+00  2.59948044e+01
  3.49594045e+00  1.84731312e+01  5.82925606e+00  5.63449383e+00
 -1.32872534e+00  4.74322701e+00  3.08122396e+00  4.26985550e+00
  4.49911356e+00  2.01676536e+00  1.23375835e+01 -2.35337782e+00
  3.71199679e+00  3.42932582e+00 -1.39722252e+00  6.31314850e+00
 -7.53959894e-01 -1.51754630e+00  5.24432993e+00 -2.86455250e+00
  9.80099106e+00  8.09895039e+00  1.29346838e+01 

In [27]:
spline_model = []

for i in range(repeat_cv):
    dat = list(zip(oof_preds[i],np.where(y>0,1,0)))
    dat = sorted(dat, key = lambda x: x[0])
    datdict = {}
    for k in range(len(dat)):
        datdict[dat[k][0]]= dat[k][1]
        
    spline_model.append(UnivariateSpline(list(datdict.keys()), list(datdict.values())))
    spline_fit = spline_model[i](oof_preds[i])
    
    print(f"logloss of cvsplit {i}: {log_loss(np.where(y>0,1,0),spline_fit)}") 

logloss of cvsplit 0: 0.5125175738238397
logloss of cvsplit 1: 0.5138533626296884
logloss of cvsplit 2: 0.5125918250887339
logloss of cvsplit 3: 0.5145289029820715
logloss of cvsplit 4: 0.5118717031213915
logloss of cvsplit 5: 0.5138111768047946
logloss of cvsplit 6: 0.5140527935535993
logloss of cvsplit 7: 0.5141136492432942
logloss of cvsplit 8: 0.5138317523543188
logloss of cvsplit 9: 0.5136670724341129


In [28]:
spline_model = []

for i in range(repeat_cv):
    dat = list(zip(oof_preds[i],np.where(y>0,1,0)))
    dat = sorted(dat, key = lambda x: x[0])
    datdict = {}
    for k in range(len(dat)):
        datdict[dat[k][0]]= dat[k][1]
    spline_model.append(UnivariateSpline(list(datdict.keys()), list(datdict.values())))
    spline_fit = spline_model[i](oof_preds[i])
    spline_fit = np.clip(spline_fit,0.025,0.975)
    
    print(f"adjusted logloss of cvsplit {i}: {log_loss(np.where(y>0,1,0),spline_fit)}") 

adjusted logloss of cvsplit 0: 0.5110061966645212
adjusted logloss of cvsplit 1: 0.5124764794761477
adjusted logloss of cvsplit 2: 0.510740765242457
adjusted logloss of cvsplit 3: 0.5132593170849638
adjusted logloss of cvsplit 4: 0.5099307928143542
adjusted logloss of cvsplit 5: 0.5127938453402021
adjusted logloss of cvsplit 6: 0.5133431370796638
adjusted logloss of cvsplit 7: 0.5123192549178317
adjusted logloss of cvsplit 8: 0.5114422121955751
adjusted logloss of cvsplit 9: 0.5120801885567579


In [ ]:
spline_model = []

for i in range(repeat_cv):
    dat = list(zip(oof_preds[i],np.where(y>0,1,0)))
    dat = sorted(dat, key = lambda x: x[0])
    datdict = {}
    for k in range(len(dat)):
        datdict[dat[k][0]]= dat[k][1]
    spline_model.append(UnivariateSpline(list(datdict.keys()), list(datdict.values())))
    spline_fit = spline_model[i](oof_preds[i])
    spline_fit = np.clip(spline_fit,0.025,0.975)
    spline_fit[(tourney_data.T1_seed==1) & (tourney_data.T2_seed==16)] = 1.0
    spline_fit[(tourney_data.T1_seed==2) & (tourney_data.T2_seed==15)] = 1.0
    spline_fit[(tourney_data.T1_seed==3) & (tourney_data.T2_seed==14)] = 1.0
    spline_fit[(tourney_data.T1_seed==4) & (tourney_data.T2_seed==13)] = 1.0
    spline_fit[(tourney_data.T1_seed==16) & (tourney_data.T2_seed==1)] = 0.0
    spline_fit[(tourney_data.T1_seed==15) & (tourney_data.T2_seed==2)] = 0.0
    spline_fit[(tourney_data.T1_seed==14) & (tourney_data.T2_seed==3)] = 0.0
    spline_fit[(tourney_data.T1_seed==13) & (tourney_data.T2_seed==4)] = 0.0
    
    print(f"adjusted logloss of cvsplit {i}: {log_loss(np.where(y>0,1,0),spline_fit)}") 

In [ ]:
val_cv = []
spline_model = []

for i in range(repeat_cv):
    dat = list(zip(oof_preds[i],np.where(y>0,1,0)))
    dat = sorted(dat, key = lambda x: x[0])
    datdict = {}
    for k in range(len(dat)):
        datdict[dat[k][0]]= dat[k][1]
    spline_model.append(UnivariateSpline(list(datdict.keys()), list(datdict.values())))
    spline_fit = spline_model[i](oof_preds[i])
    spline_fit = np.clip(spline_fit,0.025,0.975)
    # spline_fit[(tourney_data.T1_seed==1) & (tourney_data.T2_seed==16) & (tourney_data.T1_Score > tourney_data.T2_Score)] = 1.0
    # spline_fit[(tourney_data.T1_seed==2) & (tourney_data.T2_seed==15) & (tourney_data.T1_Score > tourney_data.T2_Score)] = 1.0
    # spline_fit[(tourney_data.T1_seed==3) & (tourney_data.T2_seed==14) & (tourney_data.T1_Score > tourney_data.T2_Score)] = 1.0
    # spline_fit[(tourney_data.T1_seed==4) & (tourney_data.T2_seed==13) & (tourney_data.T1_Score > tourney_data.T2_Score)] = 1.0
    # spline_fit[(tourney_data.T1_seed==16) & (tourney_data.T2_seed==1) & (tourney_data.T1_Score < tourney_data.T2_Score)] = 0.0
    # spline_fit[(tourney_data.T1_seed==15) & (tourney_data.T2_seed==2) & (tourney_data.T1_Score < tourney_data.T2_Score)] = 0.0
    # spline_fit[(tourney_data.T1_seed==14) & (tourney_data.T2_seed==3) & (tourney_data.T1_Score < tourney_data.T2_Score)] = 0.0
    # spline_fit[(tourney_data.T1_seed==13) & (tourney_data.T2_seed==4) & (tourney_data.T1_Score < tourney_data.T2_Score)] = 0.0
    
    val_cv.append(pd.DataFrame({"y":np.where(y>0,1,0), "pred":spline_fit, "season":tourney_data.Season}))
    print(f"adjusted logloss of cvsplit {i}: {log_loss(np.where(y>0,1,0),spline_fit)}") 
    
val_cv = pd.concat(val_cv)
val_cv.groupby('season').apply(lambda x: log_loss(x.y, x.pred))

In [29]:
val_cv = []
spline_model = []

for i in range(repeat_cv):
    dat = list(zip(oof_preds[i],np.where(y>0,1,0)))
    dat = sorted(dat, key = lambda x: x[0])
    datdict = {}
    for k in range(len(dat)):
        datdict[dat[k][0]]= dat[k][1]
    spline_model.append(UnivariateSpline(list(datdict.keys()), list(datdict.values())))
    spline_fit = spline_model[i](oof_preds[i])
    spline_fit = np.clip(spline_fit,0.025,0.975)
    
    val_cv.append(pd.DataFrame({"y":np.where(y>0,1,0), "pred":spline_fit, "season":tourney_data.Season}))
    print(f"adjusted logloss of cvsplit {i}: {log_loss(np.where(y>0,1,0),spline_fit)}") 
    
val_cv = pd.concat(val_cv)
val_cv.groupby('season').apply(lambda x: log_loss(x.y, x.pred))

adjusted logloss of cvsplit 0: 0.5110061966645212
adjusted logloss of cvsplit 1: 0.5124764794761477
adjusted logloss of cvsplit 2: 0.510740765242457
adjusted logloss of cvsplit 3: 0.5132593170849638
adjusted logloss of cvsplit 4: 0.5099307928143542
adjusted logloss of cvsplit 5: 0.5127938453402021
adjusted logloss of cvsplit 6: 0.5133431370796638
adjusted logloss of cvsplit 7: 0.5123192549178317
adjusted logloss of cvsplit 8: 0.5114422121955751
adjusted logloss of cvsplit 9: 0.5120801885567579


<ipython-input-29-9b93c653fa72>:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  val_cv.groupby('season').apply(lambda x: log_loss(x.y, x.pred))


season
2003    0.535308
2004    0.513371
2005    0.512771
2006    0.573797
2007    0.451186
2008    0.480414
2009    0.484470
2010    0.492734
2011    0.506600
2012    0.508857
2013    0.528881
2014    0.507851
2015    0.449796
2016    0.557843
2017    0.493377
2018    0.554450
2019    0.454967
2021    0.541969
2022    0.559915
2023    0.541057
2024    0.483038
dtype: float64

In [30]:
sub = pd.read_csv(DATA_PATH + "SampleSubmissionStage2.csv")
sub['Season'] = sub['ID'].apply(lambda x: int(x.split('_')[0]))
sub["T1_TeamID"] = sub['ID'].apply(lambda x: int(x.split('_')[1]))
sub["T2_TeamID"] = sub['ID'].apply(lambda x: int(x.split('_')[2]))
sub.head()

,ID,Pred,Season,T1_TeamID,T2_TeamID
0,2025_1101_1102,0.5,2025,1101,1102
1,2025_1101_1103,0.5,2025,1101,1103
2,2025_1101_1104,0.5,2025,1101,1104
3,2025_1101_1105,0.5,2025,1101,1105
4,2025_1101_1106,0.5,2025,1101,1106


In [31]:
sub = pd.merge(sub, season_statistics_T1, on = ['Season', 'T1_TeamID'], how = 'left')
sub = pd.merge(sub, season_statistics_T2, on = ['Season', 'T2_TeamID'], how = 'left')

sub = pd.merge(sub, glm_quality_T1, on = ['Season', 'T1_TeamID'], how = 'left')

sub = pd.merge(sub, glm_quality_T2, on = ['Season', 'T2_TeamID'], how = 'left')

sub = pd.merge(sub, seeds_T1, on = ['Season', 'T1_TeamID'], how = 'left')
sub = pd.merge(sub, seeds_T2, on = ['Season', 'T2_TeamID'], how = 'left')
sub = pd.merge(sub, last14days_stats_T1, on = ['Season', 'T1_TeamID'], how = 'left')
sub = pd.merge(sub, last14days_stats_T2, on = ['Season', 'T2_TeamID'], how = 'left')

sub["Seed_diff"] = sub["T1_seed"] - sub["T2_seed"]

sub.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,ID,Pred,Season,T1_TeamID,T2_TeamID,T1_FGMmean,T1_FGAmean,T1_FGM3mean,T1_FGA3mean,T1_ORmean,T1_Astmean,T1_TOmean,T1_Stlmean,T1_PFmean,T1_opponent_FGMmean,T1_opponent_FGAmean,T1_opponent_FGM3mean,T1_opponent_FGA3mean,T1_opponent_ORmean,T1_opponent_Astmean,T1_opponent_TOmean,T1_opponent_Stlmean,T1_opponent_Blkmean,T1_PointDiffmean,T2_FGMmean,T2_FGAmean,T2_FGM3mean,T2_FGA3mean,T2_ORmean,T2_Astmean,T2_TOmean,T2_Stlmean,T2_PFmean,T2_opponent_FGMmean,T2_opponent_FGAmean,T2_opponent_FGM3mean,T2_opponent_FGA3mean,T2_opponent_ORmean,T2_opponent_Astmean,T2_opponent_TOmean,T2_opponent_Stlmean,T2_opponent_Blkmean,T2_PointDiffmean,T1_quality,T2_quality,T1_seed,T2_seed,T1_win_ratio_14d,T2_win_ratio_14d,Seed_diff
0,2025_1101_1102,0.5,2025,1101,1102,24.206897,56.241379,4.103448,14.206897,8.655172,12.482759,14.206897,10.034483,21.448276,23.655172,51.310345,5.448276,16.689655,8.758621,12.206897,16.034483,7.896552,4.689655,-3.448276,21.250000,50.500000,8.093750,24.468750,5.937500,13.375000,12.187500,5.656250,17.843750,26.406250,54.937500,7.218750,19.906250,7.937500,12.843750,9.687500,6.812500,3.218750,-11.718750,NaN,NaN,NaN,NaN,0.333333,0.0,NaN
1,2025_1101_1103,0.5,2025,1101,1103,24.206897,56.241379,4.103448,14.206897,8.655172,12.482759,14.206897,10.034483,21.448276,23.655172,51.310345,5.448276,16.689655,8.758621,12.206897,16.034483,7.896552,4.689655,-3.448276,30.218750,64.125000,10.656250,29.093750,9.968750,17.687500,11.750000,7.562500,18.281250,26.687500,61.375000,7.437500,23.343750,8.843750,11.937500,12.000000,7.187500,2.437500,8.062500,NaN,NaN,NaN,13.0,0.333333,1.0,NaN
2,2025_1101_1104,0.5,2025,1101,1104,24.206897,56.241379,4.103448,14.206897,8.655172,12.482759,14.206897,10.034483,21.448276,23.655172,51.310345,5.448276,16.689655,8.758621,12.206897,16.034483,7.896552,4.689655,-3.448276,31.090909,64.424242,10.424242,29.787879,10.969697,16.787879,12.121212,6.060606,19.030303,28.909091,68.090909,7.363636,23.909091,10.333333,12.666667,10.060606,7.818182,4.242424,9.696970,NaN,NaN,NaN,2.0,0.333333,0.5,NaN
3,2025_1101_1105,0.5,2025,1101,1105,24.206897,56.241379,4.103448,14.206897,8.655172,12.482759,14.206897,10.034483,21.448276,23.655172,51.310345,5.448276,16.689655,8.758621,12.206897,16.034483,7.896552,4.689655,-3.448276,23.137931,59.793103,7.655172,24.862069,11.344828,12.344828,15.103448,8.000000,21.310345,26.551724,56.517241,8.275862,21.206897,9.724138,15.344828,14.137931,9.068966,3.034483,-10.586207,NaN,NaN,NaN,NaN,0.333333,0.0,NaN
4,2025_1101_1106,0.5,2025,1101,1106,24.206897,56.241379,4.103448,14.206897,8.655172,12.482759,14.206897,10.034483,21.448276,23.655172,51.310345,5.448276,16.689655,8.758621,12.206897,16.034483,7.896552,4.689655,-3.448276,25.121212,62.787879,8.848485,26.909091,9.333333,11.393939,8.727273,7.636364,18.212121,24.818182,56.757576,7.484848,22.393939,9.151515,12.484848,12.060606,5.242424,3.272727,-0.303030,NaN,NaN,NaN,16.0,0.333333,1.0,NaN


In [32]:
Xsub = sub[features].values
dtest = xgb.DMatrix(Xsub)

In [33]:
sub_models = []
for i in range(repeat_cv):
    print(f"Fold repeater {i}")
    sub_models.append(
        xgb.train(
          params = param,
          dtrain = dtrain,
          num_boost_round = int(iteration_counts[i] * 1.05),
          verbose_eval = 50
        )
    )

Fold repeater 0


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:27:40] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


Fold repeater 1


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:27:46] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


Fold repeater 2


...

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:28:26] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


Fold repeater 8


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:28:31] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


Fold repeater 9


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:28:36] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


In [34]:
sub_preds = []
for i in range(repeat_cv):
    sub_preds.append(np.clip(spline_model[i](np.clip(sub_models[i].predict(dtest),-30,30)),0.025,0.975))
    
sub["Pred"] = pd.DataFrame(sub_preds).mean(axis=0)
sub[['ID','Pred']].to_csv("submission.csv", index = None)

In [35]:
subb = pd.read_csv("/kaggle/working/submission.csv")
subb.tail()

,ID,Pred
131402,2025_3477_3479,0.176091
131403,2025_3477_3480,0.088492
131404,2025_3478_3479,0.155267
131405,2025_3478_3480,0.070716
131406,2025_3479_3480,0.047002
